<a href="https://colab.research.google.com/github/mrc329/datakit-financial-inclusion-2025/blob/main/SQL_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Download data files if needed
!wget -O kenya_census_cleaned.csv https://raw.githubusercontent.com/mrc329/sql-portfolio/main/kenya_census_cleaned.csv
!wget -O banking_data_by_county_kenya.csv https://raw.githubusercontent.com/mrc329/sql-portfolio/main/banking_data_by_county_kenya.csv

# Load CSVs into pandas DataFrames
df_census = pd.read_csv("kenya_census_cleaned.csv")
df_banking = pd.read_csv("banking_data_by_county_kenya.csv")

# Clean and normalize column names
df_census.columns = df_census.columns.str.strip().str.lower()
df_banking.columns = df_banking.columns.str.strip().str.lower()

# Create database connection
conn = sqlite3.connect('my_data.db')

# Create tables in database
df_census.to_sql('census_data', conn, if_exists='replace', index=False)
df_banking.to_sql('banking_data', conn, if_exists='replace', index=False)

# Function to run SQL queries
def run_query(query):
    return pd.read_sql_query(query, conn)

# Test the connection with a sample query
print("Sample data from census_data table:")
print(run_query("SELECT * FROM census_data LIMIT 5;"))

print("\nSample data from banking_data table:")
print(run_query("SELECT * FROM banking_data LIMIT 5;"))

# Now you can run your SQL queries using the run_query function
# For example:
# result = run_query("SELECT * FROM banking_data WHERE bank_overall > 40;")

# Remember to close the connection when done
# conn.close()

--2025-04-29 12:51:27--  https://raw.githubusercontent.com/mrc329/sql-portfolio/main/kenya_census_cleaned.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5078 (5.0K) [text/plain]
Saving to: ‘kenya_census_cleaned.csv’

kenya_census_cleane 100%[===================>]   4.96K  --.-KB/s    in 0s      

2025-04-29 12:51:28 (48.7 MB/s) - ‘kenya_census_cleaned.csv’ saved [5078/5078]

--2025-04-29 12:51:28--  https://raw.githubusercontent.com/mrc329/sql-portfolio/main/banking_data_by_county_kenya.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 20

In [ ]:
%load_ext sql
%sql sqlite:///my_data.db

In [ ]:
%%sql
-- Census names with no banking match
SELECT DISTINCT c.county
FROM census AS c
LEFT JOIN banking_data AS b
  ON UPPER(TRIM(c.county)) = UPPER(TRIM(b.county))
WHERE b.county IS NULL;

 * sqlite:///my_data.db
(sqlite3.OperationalError) no such table: census
[SQL: -- Census names with no banking match
SELECT DISTINCT c.county
FROM census AS c
LEFT JOIN banking_data AS b
  ON UPPER(TRIM(c.county)) = UPPER(TRIM(b.county))
WHERE b.county IS NULL;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
final_results=(run_query("""
WITH joined_data AS (
  SELECT *
  FROM census_data AS c
  JOIN banking_data AS b
    ON lower(trim(replace(replace(replace(c.county, '/', ' '), '-', ' '), '''', '')))
    = lower(trim(replace(replace(replace(b.county, '/', ' '), '-', ' '), '''', '')))
),

final_metrics AS (
  SELECT
    county,
    -- Formal_Financial_Inclusion
    ROUND(CAST(REPLACE(total_pop, ',', '') AS FLOAT) / bank_overall, 1) AS people_banked,
    printf('%.6f', bank_overall * 1.0 / CAST(REPLACE(total_pop, ',', '') AS FLOAT)) AS banks_per_person,
    printf('%.6f', bank_overall * 1.0 / CAST(REPLACE(density_persons_per_sq_km, ',', '') AS FLOAT)) AS bank_per_population_density,
CASE
  WHEN bank_overall IS NULL OR bank_overall = 0 THEN NULL
  ELSE ROUND(
    (CAST(REPLACE(bank_overall, ',', '') AS FLOAT) / male_pop) / (CAST(REPLACE(bank_overall, ',', '') AS FLOAT) / female_pop),
    2
  )
END AS gender_banking_gap,


    -- Banking_per_Household_Type
    printf('%.6f', bank_overall * 1.0 / CAST(REPLACE(total_households, ',', '') AS FLOAT)) AS bank_to_household_ratio,
    ROUND(CAST(REPLACE(total_households, ',', '') AS FLOAT) / bank_overall, 2) AS households_per_bank_account,
    printf('%.6f', bank_overall * 1.0 / CAST(REPLACE(conventional_households, ',', '') AS FLOAT)) AS bank_to_conventional_household_ratio,
    ROUND(CAST(REPLACE(conventional_households, ',', '') AS FLOAT) / bank_overall, 2) AS conventional_households_per_bank_account,
    printf('%.6f', bank_overall * 1.0 / CAST(REPLACE(group_quarters_households, ',', '') AS FLOAT)) AS bank_to_group_household_ratio,
    ROUND(CAST(REPLACE(group_quarters_households, ',', '') AS FLOAT) / bank_overall, 2) AS group_households_per_bank_account,

    -- Community_Financial_Access
    ROUND(CAST(REPLACE(sacco, ',', '') AS FLOAT) / total_pop, 2) AS sacco_per_person,
    ROUND(CAST(REPLACE(total_pop, ',', '') AS FLOAT) / sacco, 2) AS people_per_sacco,
    printf('%.6f', sacco * 1.0 / CAST(REPLACE(density_persons_per_sq_km, ',', '') AS FLOAT)) AS sacco_per_population_density,
    CASE
  WHEN sacco IS NULL OR sacco = 0 THEN NULL
  ELSE ROUND(
    (CAST(REPLACE(sacco, ',', '') AS FLOAT) / male_pop) / (CAST(REPLACE(sacco, ',', '') AS FLOAT) / female_pop),
    2
  )
END AS sacco_gender_gap,

    -- Digital/Mobile_Money_Access
    printf('%.6f', mobile_bank_account * 1.0 / CAST(REPLACE(total_pop, ',', '') AS FLOAT)) AS mobile_accts_per_person,
    ROUND(CAST(REPLACE(total_pop, ',', '') AS FLOAT) / mobile_bank_account, 2) AS people_per_mobile_acct,
    printf('%.6f', mobile_money * 1.0 / CAST(REPLACE(density_persons_per_sq_km, ',', '') AS FLOAT)) AS mobile_money_per_population_density,
    ROUND(CAST(REPLACE(mobile_money, ',', '') AS FLOAT) / total_pop, 2) AS mobile_money_per_person,
    ROUND(CAST(REPLACE(total_pop, ',', '') AS FLOAT) / mobile_money, 2) AS people_per_mobile_money,
       CASE
  WHEN mobile_bank_account IS NULL OR mobile_bank_account = 0 THEN NULL
  ELSE ROUND(
    (CAST(REPLACE(mobile_bank_account, ',', '') AS FLOAT) / male_pop) / (CAST(REPLACE(mobile_bank_account, ',', '') AS FLOAT) / female_pop),
    2
  )
END AS mobile_bank_account_gender_gap,
    printf('%.6f', mobile_bank_account * 1.0 / CAST(REPLACE(density_persons_per_sq_km, ',', '') AS FLOAT)) AS mobile_bank_account_per_population_density,
         CASE
  WHEN mobile_money IS NULL OR mobile_money = 0 THEN NULL
  ELSE ROUND(
    (CAST(REPLACE(mobile_money, ',', '') AS FLOAT) / male_pop) / (CAST(REPLACE(mobile_money, ',', '') AS FLOAT) / female_pop),
    2
  )
END AS mobile_money_gender_gap,


    -- Micro_Finance
    ROUND(CAST(REPLACE(micro_finance, ',', '') AS FLOAT) / total_pop, 2) AS micro_finance_per_person,
    ROUND(CAST(REPLACE(total_pop, ',', '') AS FLOAT) / micro_finance, 2) AS people_per_micro_finance,
  CASE
  WHEN micro_finance IS NULL OR micro_finance = 0  THEN 0
ELSE ROUND(
    (CAST(REPLACE(micro_finance, ',', '') AS FLOAT) / male_pop) / (CAST(REPLACE(micro_finance, ',', '') AS FLOAT) / female_pop),
    2
  )  END AS micro_finance_gender_gap,


    -- Financial_Resilience
    ROUND(CAST(REPLACE(total_pop, ',', '') AS FLOAT) / insurance, 2) AS people_per_insurance,
    ROUND(CAST(REPLACE(insurance, ',', '') AS FLOAT) / total_pop, 2) AS insurance_per_person,
    ROUND(CAST(REPLACE(total_pop, ',', '') AS FLOAT) / pension, 2) AS people_per_pension,
    ROUND(CAST(REPLACE(pension, ',', '') AS FLOAT) / total_pop, 2) AS pension_per_person,
    CASE
      WHEN insurance IS NULL OR insurance = 0 THEN NULL
      ELSE printf('%.6f', (CAST(REPLACE(insurance, ',', '') AS FLOAT) / male_pop) / (CAST(REPLACE(insurance, ',', '') AS FLOAT) / female_pop))
    END AS insurance_gender_gap,
    CASE
      WHEN pension IS NULL OR pension = 0 THEN NULL
      ELSE printf('%.6f', (CAST(REPLACE(pension, ',', '') AS FLOAT) / male_pop) / (CAST(REPLACE(pension, ',', '') AS FLOAT) / female_pop))
    END AS pension_gender_gap,

    -- Household_Metrics
    ROUND(CAST(REPLACE(sacco, ',', '') AS FLOAT) / total_households, 2) AS sacco_per_household,
    ROUND(CAST(REPLACE(total_households, ',', '') AS FLOAT) / sacco, 2) AS household_per_sacco,
    ROUND(CAST(REPLACE(micro_finance, ',', '') AS FLOAT) / total_households, 2) AS micro_finance_per_household,
   CASE
  WHEN micro_finance IS NULL OR micro_finance = 0 OR total_households IS NULL THEN 0
  ELSE ROUND(CAST(REPLACE(total_households, ',', '') AS FLOAT) / micro_finance, 2)
END AS household_per_micro_finance,
    ROUND(CAST(REPLACE(insurance, ',', '') AS FLOAT) / total_households, 2) AS insurance_per_household,
    ROUND(CAST(REPLACE(total_households, ',', '') AS FLOAT) / insurance, 2) AS household_per_insurance,
    ROUND(CAST(REPLACE(pension, ',', '') AS FLOAT) / total_households, 2) AS pension_per_household,
    ROUND(CAST(REPLACE(total_households, ',', '') AS FLOAT) / pension, 2) AS household_per_pension
  FROM joined_data
),

ntile_metrics AS (
  SELECT
    *,
    NTILE(4) OVER (ORDER BY people_per_insurance) AS people_per_insurance_quartile,
    NTILE(4) OVER (ORDER BY insurance_per_person) AS insurance_per_person_quartile,
    NTILE(4) OVER (ORDER BY people_per_pension) AS people_per_pension_quartile,
    NTILE(4) OVER (ORDER BY pension_per_person) AS pension_per_person_quartile,
    NTILE(4) OVER (ORDER BY insurance_gender_gap) AS insurance_gender_gap_quartile,
    NTILE(4) OVER (ORDER BY pension_gender_gap) AS pension_gender_gap_quartile,
    NTILE(4) OVER (ORDER BY sacco_per_household) AS sacco_per_household_quartile,
    NTILE(4) OVER (ORDER BY household_per_sacco) AS household_per_sacco_quartile,
    NTILE(4) OVER (ORDER BY micro_finance_per_household) AS micro_finance_per_household_quartile,
    NTILE(4) OVER (ORDER BY household_per_micro_finance) AS household_per_micro_finance_quartile,
    NTILE(4) OVER (ORDER BY micro_finance_per_person) AS micro_finance_per_person_quartile,
    NTILE(4) OVER (ORDER BY insurance_per_household) AS insurance_per_household_quartile,
    NTILE(4) OVER (ORDER BY household_per_insurance) AS household_per_insurance_quartile,
    NTILE(4) OVER (ORDER BY pension_per_household) AS pension_per_household_quartile,
    NTILE(4) OVER (ORDER BY household_per_pension) AS household_per_pension_quartile
  FROM final_metrics
),
calculated_metrics AS (
    SELECT *,
        (people_banked + sacco_per_person + mobile_accts_per_person + micro_finance_per_person) / 4 AS Financial_Access_Index_FAI,
        (gender_banking_gap + sacco_gender_gap + mobile_bank_account_gender_gap + mobile_money_gender_gap + micro_finance_gender_gap + insurance_gender_gap + pension_gender_gap) / 7 AS Gender_Equality_Score,
        (households_per_bank_account + household_per_sacco + household_per_micro_finance + household_per_insurance + household_per_pension) / 5 AS Household_Financial_Access_Index,
        (bank_per_population_density + sacco_per_population_density + mobile_bank_account_per_population_density + mobile_money_per_population_density) / 4 AS Density_Adjusted_Access_Score,
        (insurance_per_person + pension_per_person) / 2 AS Resilance_Score,
        (banks_per_person + sacco_per_person) / (mobile_accts_per_person + mobile_money_per_person) AS Traditional_vs_Digital_Finance_Balance
    FROM ntile_metrics
)
SELECT *,
    (Financial_Access_Index_FAI - MIN(Financial_Access_Index_FAI) OVER ()) /
        NULLIF((MAX(Financial_Access_Index_FAI) OVER () - MIN(Financial_Access_Index_FAI) OVER ()), 0) AS Financial_Access_Index_FAI_norm,
    (Gender_Equality_Score - MIN(Gender_Equality_Score) OVER ()) /
        NULLIF((MAX(Gender_Equality_Score) OVER () - MIN(Gender_Equality_Score) OVER ()), 0) AS Gender_Equality_Score_norm,
    (Household_Financial_Access_Index - MIN(Household_Financial_Access_Index) OVER ()) /
        NULLIF((MAX(Household_Financial_Access_Index) OVER () - MIN(Household_Financial_Access_Index) OVER ()), 0) AS Household_Financial_Access_Index_norm,
    (Density_Adjusted_Access_Score - MIN(Density_Adjusted_Access_Score) OVER ()) /
        NULLIF((MAX(Density_Adjusted_Access_Score) OVER () - MIN(Density_Adjusted_Access_Score) OVER ()), 0) AS Density_Adjusted_Access_Score_norm,
    (Resilance_Score - MIN(Resilance_Score) OVER ()) /
        NULLIF((MAX(Resilance_Score) OVER () - MIN(Resilance_Score) OVER ()), 0) AS Resilance_Score_norm,
    (Traditional_vs_Digital_Finance_Balance - MIN(Traditional_vs_Digital_Finance_Balance) OVER ()) /
        NULLIF((MAX(Traditional_vs_Digital_Finance_Balance) OVER () - MIN(Traditional_vs_Digital_Finance_Balance) OVER ()), 0) AS Traditional_vs_Digital_Finance_Balance_norm
FROM calculated_metrics;
"""))

In [ ]:
from google.colab import files
df.to_csv('final_metric', index=False)
files.download('financial_metrics_analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%config SqlMagic.style = 'DEFAULT'
%reload_ext sql
%sql sqlite:///my_data.db
